In [1]:
import os
import time
import scipy
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import numpy as np
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
tfk = tf.keras
tfkl = tf.keras.layers

In [2]:
onColab = True
if onColab:
  from google.colab import drive
  drive.mount('/gdrive')
  %cd /gdrive/MyDrive/University/ANN/CHALLENGE1/with_img_divided
  !ls

Mounted at /gdrive
/gdrive/.shortcut-targets-by-id/14nAfiUIlOkzENFAyByV5FH1KxQM8uSlm/CHALLENGE1/with_img_divided
dataset.zip  on_new_ds_new_pipeline.ipynb


In [3]:
doUnzipData = True
batch_size = 256
validation_split = 0.2

if doUnzipData:
  !unzip dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: training/Tomato_others/37908.jpg  
  inflating: training/Tomato_others/37909.jpg  
  inflating: training/Tomato_others/37911.jpg  
  inflating: training/Tomato_others/37912.jpg  
  inflating: training/Tomato_others/37913.jpg  
  inflating: training/Tomato_others/37914.jpg  
  inflating: training/Tomato_others/37916.jpg  
  inflating: training/Tomato_others/37917.jpg  
  inflating: training/Tomato_others/37920.jpg  
  inflating: training/Tomato_others/37921.jpg  
  inflating: training/Tomato_others/37922.jpg  
  inflating: training/Tomato_others/37923.jpg  
  inflating: training/Tomato_others/37924.jpg  
  inflating: training/Tomato_others/37925.jpg  
  inflating: training/Tomato_others/37926.jpg  
  inflating: training/Tomato_others/37927.jpg  
  inflating: training/Tomato_others/37928.jpg  
  inflating: training/Tomato_others/37929.jpg  
  inflating: training/Tomato_others/37931.jpg  
  inflating: training/Tomato_others/3

In [4]:
#@title init seed everywhere
seed =20

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [5]:
dataset_path = './training'
img_h, img_w = (256, 256)
test_path = './test'


def load_data(dontUseFun=True, fun=None, isTest = False):
  """
  load data (train, val, test)
  dontUseFun: if it's true then the data is not preprocessed. If it's false, the data is preprocessed with fun (which must not be none)
  """
  image_generator = ImageDataGenerator(preprocessing_function = fun, validation_split=validation_split, 
                                          rotation_range=30,
                                          height_shift_range=50,
                                          width_shift_range=50,
                                          zoom_range=0.3,
                                          horizontal_flip=True,
                                          vertical_flip=True, 
                                          fill_mode='reflect')  
  if dontUseFun:
    image_generator = ImageDataGenerator(validation_split=validation_split, 
                                          rotation_range=30,
                                          height_shift_range=50,
                                          width_shift_range=50,
                                          zoom_range=0.3,
                                          horizontal_flip=True,
                                          vertical_flip=True, 
                                          fill_mode='reflect')  


  # Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
  train_gen = image_generator.flow_from_directory(directory=dataset_path,
                                                target_size=(img_h, img_w),
                                                color_mode='rgb',
                                                classes=None, # can be set to None
                                                class_mode='categorical',
                                                subset='training',
                                                batch_size=batch_size,
                                                shuffle=True,
                                                seed=seed)

  valid_gen = image_generator.flow_from_directory(directory=dataset_path,
                                                target_size=(img_h, img_w),
                                                color_mode='rgb',
                                                classes=None, # can be set to None
                                                class_mode='categorical',
                                                subset='validation',
                                                batch_size=batch_size,
                                                shuffle=False,
                                                seed=seed)
  test_gen = None
  if isTest:
    test_image_gen = ImageDataGenerator(preprocessing_function = fun)
    if dontUseFun:
      test_image_gen = ImageDataGenerator(rotation_range=30)
    
    test_gen = test_image_gen.flow_from_directory(directory=test_path,
                                                target_size=(img_h, img_w),
                                                color_mode='rgb',
                                                classes=None, # can be set to None
                                                class_mode='categorical',
                                                batch_size=batch_size,
                                                shuffle=True, # sometimes, instead of train, we use the test for training
                                                seed=seed)
  return train_gen, valid_gen, test_gen


In [14]:
train_gen = load_data(False, tl_preprocess, False)[0]

Found 14058 images belonging to 23 classes.
Found 3502 images belonging to 23 classes.


In [24]:
labels = list(train_gen.class_indices.keys())
print(labels)

['Apple', 'Apple_ill', 'Blueberry', 'Cherry', 'Cherry_ill', 'Corn_in', 'Corn_out', 'Grape', 'Grape_ill', 'Orange', 'Peach', 'Peach_ill', 'Pepper', 'Pepper_ill', 'Potato', 'Potato_ill', 'Raspberry', 'Soybean', 'Squash', 'Strawberry', 'Strawberry_ill', 'Tomato_green', 'Tomato_others']


In [34]:
create_test_set = False

# IF YOU CHANGE NET GO CHECK THE FINE TUNING CODE IF IT IS ALRIGHT!

model_dir = 'ResNet50_best_128_denselayerft_new_DS_weightOnClasses'

# Highly dependent on the network chosen:
n_not_trainable_layers = 169

net = tfk.applications.ResNet50


def tl_preprocess(x):
  return tf.keras.applications.resnet50.preprocess_input(x)


print(labels)

def compute_weight(val_split):
  w = []
  for i in range(len(labels)):
    class_imgs = next(os.walk('{}/{}/'.format('./training', labels[i])))[2]
    w.append(len(class_imgs) * (1 - val_split))

  tot = sum(w)
  for i in range(len(w)):
    w[i] = tot / w[i]
  m = max(w)
  w = np.array(w) / m
  w[np.argmax(w)] = 0.99999999999999
  d = {}
  for i in range(len(w)):
    d[i] = w[i]
  return d

# weights computed for validation split = 0.3
class_weight = {0: 0.1741682974559687, 1: 0.18936170212765957, 2: 0.1913978494623656, 3: 0.3423076923076923, 4: 0.27899686520376177, 5: 0.24929971988795518, 6: 0.10607866507747318, 7: 0.2908496732026144, 8: 0.0778652668416448, 9: 0.051061388410786, 10: 0.34765625, 11: 0.12378303198887343, 12: 0.1943231441048035, 13: 0.2947019867549669, 14: 0.99999999999999, 15: 0.14217252396166136, 16: 0.3371212121212121, 17: 0.05517668939863608, 18: 0.16729323308270677, 19: 0.2680722891566265, 20: 0.27217125382262997, 21: 0.1790744466800805, 22: 0.017342166796570538}
mappings = {
    0: 0, 1: 0,
    2: 1,
    3: 2, 4: 2,
    5: 3, 6: 3,
    7: 4, 8: 4,
    9: 5,
    10: 6, 11: 6,
    12: 7, 13: 7,
    14: 8, 15: 8,
    16: 9, 
    17: 10,
    18: 11,
    19: 12, 20: 12,
    21: 13, 22: 13
}
drop_out_last_dense = 0.3
fc_dense_n_neurons = 128

comment = 'resnet50, transfer learning, fine tuned. dense layer with drop out, 128 neurons. test of 210 imgs (first 15 imgs per class).'
comment += '\nnot trainable layers: ' + str(n_not_trainable_layers)
comment += ' drop_out_last_dense ' + str(drop_out_last_dense)
comment += ' fc_dense_n_neurons ' + str(fc_dense_n_neurons)
comment += '\nvalidation_split = ' + str(validation_split) 
comment += '. batch_size = ' + str(batch_size) 
comment += '. seed = ' + str(seed)
comment += '\nweigths: '
comment += str(class_weight)
print(comment)

['Apple', 'Apple_ill', 'Blueberry', 'Cherry', 'Cherry_ill', 'Corn_in', 'Corn_out', 'Grape', 'Grape_ill', 'Orange', 'Peach', 'Peach_ill', 'Pepper', 'Pepper_ill', 'Potato', 'Potato_ill', 'Raspberry', 'Soybean', 'Squash', 'Strawberry', 'Strawberry_ill', 'Tomato_green', 'Tomato_others']
resnet50, transfer learning, fine tuned. dense layer with drop out, 128 neurons. test of 210 imgs (first 15 imgs per class).
not trainable layers: 169 drop_out_last_dense 0.3 fc_dense_n_neurons 128
validation_split = 0.2. batch_size = 256. seed = 20
weigths: {0: 0.1741682974559687, 1: 0.18936170212765957, 2: 0.1913978494623656, 3: 0.3423076923076923, 4: 0.27899686520376177, 5: 0.24929971988795518, 6: 0.10607866507747318, 7: 0.2908496732026144, 8: 0.0778652668416448, 9: 0.051061388410786, 10: 0.34765625, 11: 0.12378303198887343, 12: 0.1943231441048035, 13: 0.2947019867549669, 14: 0.99999999999999, 15: 0.14217252396166136, 16: 0.3371212121212121, 17: 0.05517668939863608, 18: 0.16729323308270677, 19: 0.2680722

In [27]:
if create_test_set:
  import shutil
  os.mkdir(test_path)
  for i in range(len(labels)):
      class_imgs = next(os.walk('{}/{}/'.format(dataset_dir, labels[i])))[2]
      # In this way we always get the same data. 
      # TODO: randomize using seed the data we take
      class_imgs.sort()
      for j in range(5):
        class_img = class_imgs[j]
        dest_dir = './test/' + labels[i] + '/'
        if not os.path.exists(dest_dir):
          os.mkdir(dest_dir)
        shutil.move('{}/{}/'.format(dataset_dir, labels[i]) + class_img, dest_dir)


In [28]:
def preprocess(x):
  # x = clean_image(x)
  if tl_preprocess:
    return tl_preprocess(np.uint8(x))
  return x

In [29]:
train_gen, valid_gen, test_gen = load_data(False, preprocess)

Found 14058 images belonging to 23 classes.
Found 3502 images belonging to 23 classes.


In [30]:
def get_next_batch(generator):
  batch = next(generator)

  image = batch[0]
  target = batch[1]

  print("(Input) image shape:", image.shape)
  print("Target shape:",target.shape)

  # Visualize only the first sample
  image = image[0]
  target = target[0]
  target_idx = np.argmax(target)
  print()
  print("Categorical label:", target)
  print("Label:", target_idx)
  print("Class name:", labels[target_idx])
  fig = plt.figure(figsize=(6, 4))
  plt.imshow(image)
  plt.title("image which will be fed to the cnn")
  return batch

# Get a sample from dataset and show info
# _ = get_next_batch(train_gen)

In [31]:
supernet =  net(
    weights="imagenet",
    input_shape=(img_h, img_w, 3),
    include_top=False
    )

94781440/94765736 [==============================] - 1s 0us/step


In [32]:
print(len(supernet.layers))

175


In [35]:
# let last block trainable
for i, layer in enumerate(supernet.layers[:n_not_trainable_layers]):
  layer.trainable=False
for i, layer in enumerate(supernet.layers):
  print(i, layer.name, layer.trainable)
# supernet.summary()

# Now we use the supernet as layer of our network
inputs = tfk.Input(shape=(img_h, img_w, 3))
x = supernet(inputs)

# Now we build our classifier (which will be actually trained)
x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(drop_out_last_dense, seed=seed)(x)
x = tfkl.Dense(
    fc_dense_n_neurons, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

# x = tfkl.Dropout(0.3, seed=seed)(x)
outputs = tfkl.Dense(
    len(labels), 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

model.summary()

0 input_1 False
1 conv1_pad False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 pool1_pad False
6 pool1_pool False
7 conv2_block1_1_conv False
8 conv2_block1_1_bn False
9 conv2_block1_1_relu False
10 conv2_block1_2_conv False
11 conv2_block1_2_bn False
12 conv2_block1_2_relu False
13 conv2_block1_0_conv False
14 conv2_block1_3_conv False
15 conv2_block1_0_bn False
16 conv2_block1_3_bn False
17 conv2_block1_add False
18 conv2_block1_out False
19 conv2_block2_1_conv False
20 conv2_block2_1_bn False
21 conv2_block2_1_relu False
22 conv2_block2_2_conv False
23 conv2_block2_2_bn False
24 conv2_block2_2_relu False
25 conv2_block2_3_conv False
26 conv2_block2_3_bn False
27 conv2_block2_add False
28 conv2_block2_out False
29 conv2_block3_1_conv False
30 conv2_block3_1_bn False
31 conv2_block3_1_relu False
32 conv2_block3_2_conv False
33 conv2_block3_2_bn False
34 conv2_block3_2_relu False
35 conv2_block3_3_conv False
36 conv2_block3_3_bn False
37 conv2_block3_add False
38 conv2_bloc

In [ ]:
c = [tfk.callbacks.ModelCheckpoint('./ckpt_' + model_dir), tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)] 
history = model.fit(
    x = train_gen,
    batch_size = batch_size,
    epochs = 200,
    validation_data = valid_gen,
    class_weight = class_weight,
    callbacks = c
).history

Epoch 1/200
55/55 [==============================] - ETA: 0s - loss: 0.4711 - accuracy: 0.4308INFO:tensorflow:Assets written to: ./ckpt_ResNet50_best_128_denselayerft_new_DS_weightOnClasses/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


55/55 [==============================] - 450s 7s/step - loss: 0.4711 - accuracy: 0.4308 - val_loss: 1.2036 - val_accuracy: 0.6630
Epoch 2/200
55/55 [==============================] - ETA: 0s - loss: 0.0638 - accuracy: 0.7921

In [ ]:
model.save('./' + model_dir)

In [ ]:
# model.evaluate(test_gen, return_dict=True)

In [ ]:
def compareModels(history1, h1lbl, history2=None, h2lbl=''):
  '''
  Plot history1 against history2. 
  If history2 is None then Plot twice history1 (it'll be overlapped)
  h1lbl and h2lbl are the labels of the two histories
  '''
  if history2 == None: 
    history2 = history1
    h2lbl = h1lbl
  # Plot the training
  plt.figure(figsize=(15,5))
  plt.plot(history1['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
  plt.plot(history1['val_loss'], label=h1lbl, alpha=.8, color='#ff7f0e')
  plt.plot(history2['loss'], alpha=.3, color='#4D61E2', linestyle='--')
  plt.plot(history2['val_loss'], label=h2lbl, alpha=.8, color='#4D61E2')
  plt.legend(loc='upper left')
  plt.title('Categorical Crossentropy')
  plt.grid(alpha=.3)

  plt.figure(figsize=(15,5))
  plt.plot(history1['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
  plt.plot(history1['val_accuracy'], label=h1lbl, alpha=.8, color='#ff7f0e')
  plt.plot(history2['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
  plt.plot(history2['val_accuracy'], label=h2lbl, alpha=.8, color='#4D61E2')
  plt.legend(loc='upper left')
  plt.title('Accuracy')
  plt.grid(alpha=.3)

  plt.show()
compareModels(history, model_dir)

In [ ]:
model = tf.keras.models.load_model(os.path.join('./', model_dir))

In [ ]:
# # Predict the test set with the CNN
# X_test = []
# y_test = []
# n = next(test_gen)
# while n:
#   X_test.append(n[0])
#   y_test.append(n[1])
#   n = next(test_gen)

# predictions = model.predict(X_test)
# predictions.shape

# # Compute the confusion matrix
# cm = confusion_matrix(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))

# # Compute the classification metrics
# accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))
# precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
# recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
# f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
# print('Accuracy:',accuracy.round(4))
# print('Precision:',precision.round(4))
# print('Recall:',recall.round(4))
# print('F1:',f1.round(4))

# # Plot the confusion matrix
# plt.figure(figsize=(10,8))
# sns.heatmap(cm.T, xticklabels=list(labels.values()), yticklabels=list(labels.values()))
# plt.xlabel('True labels')
# plt.ylabel('Predicted labels')
# plt.show()

In [ ]:
# TODO: compute confusion
# from sklearn.metrics import confusion_matrix


# cm = confusion_matrix(
#   np.concatenate([np.argmax(test_gen[i][1], axis=1) for i in range(n_batches)]),    
#   np.argmax(model.predict(test_gen, steps=n_batches), axis=1) 
# )
# print(cm)

In [ ]:
# # Predict the test set with the CNN
# from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
# n_batches = 11

# predictions = model.predict(test_gen, steps=n_batches)
# predictions.shape

# target = np.concatenate([np.argmax(test_gen[i][1], axis=1) for i in range(n_batches)])

# # Compute the confusion matrix
# cm = confusion_matrix(target, np.argmax(predictions, axis=1))
# print(cm)



In [ ]:
# # # Compute the classification metrics
# accuracy = accuracy_score(target, np.argmax(predictions, axis=1))
# precision = precision_score(target, np.argmax(predictions, axis=1), average='macro')
# recall = recall_score(target, np.argmax(predictions, axis=1), average='macro')
# f1 = f1_score(target, np.argmax(predictions, axis=1), average='macro')
# print('Accuracy:',accuracy.round(4))
# print('Precision:',precision.round(4))
# print('Recall:',recall.round(4))
# print('F1:',f1.round(4))

# # Plot the confusion matrix
# plt.figure(figsize=(10,8))
# import seaborn as sns

# sns.heatmap(cm.T, xticklabels=labels, yticklabels=labels)
# plt.xlabel('True labels')
# plt.ylabel('Predicted labels')
# plt.show()